In [ ]:
import bgk
import matplotlib.pyplot as plt
import bgk.run_params as rp
import os

In [ ]:
# Parameters to adjust figures
case = "exact" # exact, max, exact-rev, or max-rev
cont = True

nframes = 100
param = rp.ne

In [ ]:
basepath = f"/mnt/lustre/IAM851/jm1667/psc-runs/case1/trials/{case}/"
paths = ["B00.10-n256", "B00.25-n256", "B00.50-n256-cont", "B01.00-n256-cont2"]
Bs = [.1, .25, .5, 1]
ress = [256, 256, 256, 256]
maxsteps = [500000, 1000000, 2000000, 4000000]

In [ ]:
# path_fmt = f"/mnt/lustre/IAM851/jm1667/psc-runs/case1/trials/{case}/B{{:05.2f}}-n{{}}/"

def get_line(path, res, maxstep, nframes, param) -> tuple[list[float], list[float]]:
    # path = path_fmt.format(B, res)

    loader = bgk.Loader(path, engine="pscadios2", species_names=['e', 'i'], max_step=maxstep)
    videoMaker = bgk.VideoMaker(nframes, loader)

    wholeSlice = bgk.DataSlice(slice(None, None), "")
    videoMaker.loadData(param)
    videoMaker.setSlice(wholeSlice)
    
    return videoMaker.times, videoMaker._getMeansAtOrigin(res//128)

In [ ]:
# Load Data
# liness: list[list[tuple]] = []
lines: list[tuple] = []
for halfpath, res, maxstep in zip(paths, ress, maxsteps):
    path = basepath + halfpath
    print("Loading " + path + "...")
    x, y = get_line(path, res, maxstep, nframes, param)
    lines.append((x, y))

In [ ]:
# Generate Figure
fig, ax = plt.subplots()

ax.set_xlabel("Time")
ax.set_ylabel(f"Mean {param.title}")
ax.set_title(f"Comparisons of Mean {param.title} Near Origin")

for B, res, (x, y) in zip(Bs, ress, lines):
    ax.plot(x, y, label=f"$B_0={B}, n={res}$")


ax.legend();

In [ ]:
def get_fig_path(outdir, fig_name: str) -> str:
    return os.path.join(outdir, fig_name)

def get_fig_name() -> str:
    param_str = param.title.replace("_", "").replace("$", "")
    Bs_str = ",".join([f"B{B:05.2f}" for B in Bs])
    return f"scaling-{param_str}-{case}-{Bs_str}.png"

In [ ]:
# Save Figure
maybe_cont = "-cont" if cont else ""
outdir = f"/mnt/lustre/IAM851/jm1667/psc-scrap/figs-{case}{maybe_cont}/"
os.makedirs(outdir, exist_ok=True)

fig.savefig(get_fig_path(outdir, get_fig_name()), bbox_inches="tight", pad_inches=0.01, dpi=300)